In [ ]:
# In terminal (run this in your project root)
bun init

This creates a `package.json` file for managing dependencies (compatible with Bun).

## Step 3: Install Tailwind CSS and Dependencies
Install Tailwind and PostCSS for CSS processing:

In [ ]:
# Install as dev dependencies
bun add -d tailwindcss postcss autoprefixer

## Step 4: Create Tailwind and PostCSS Config Files
Tailwind CSS v4+ no longer provides an `init` command. You must create the configuration files manually.

Create a `tailwind.config.js` file in your project root with:

```js
// tailwind.config.js
module.exports = {
  content: [
    './apps/**/*.html',
    './templates/**/*.html',
    './apps/**/*.py',
  ],
  theme: {
    extend: {},
  },
  plugins: [],
}

Create a `postcss.config.js` file in your project root with:

```js
// postcss.config.js
module.exports = {
  plugins: {
    tailwindcss: {},
    autoprefixer: {},
  },
}

You can create these files using your editor or with:
```sh
touch tailwind.config.js postcss.config.js
Then paste the above content into each file.

## Step 5: Configure Tailwind
(Already covered in the config file above.)

## Step 6: Create a CSS File with Tailwind Directives
Create `static/css/tailwind.css` in your project:

```css
/* static/css/tailwind.css */
@tailwind base;
@tailwind components;
@tailwind utilities;

## Step 7: Build CSS with Bun
Use Bun to build your CSS for development (watch mode) or production (minified):

```json
// package.json (under "scripts")
{
  "scripts": {
    "build-css": "tailwindcss -i ./static/css/tailwind.css -o ./static/css/output.css --watch",
    "build-css-prod": "tailwindcss -i ./static/css/tailwind.css -o ./static/css/output.css --minify"
  }
}

## Step 8: Integrate with Django
- Link the CSS in your templates (e.g., `apps/core/templates/includes/layout.html`):

```html
<head>
    {% load static %}
    <link rel="stylesheet" href="{% static 'css/output.css' %}">
</head>

- Ensure `STATIC_URL = 'static/'` in `config/settings.py`.

## Step 9: Test and Build
Run the build command and test:

In [ ]:
# Build CSS (run in terminal)
bunx tailwindcss -i ./static/css/tailwind.css -o ./static/css/output.css --watch

# Start Django server to test
python manage.py runserver

Use Tailwind classes like `bg-blue-500` in your templates. For production, run `bunx tailwindcss -i ./static/css/tailwind.css -o ./static/css/output.css --minify`.

## File Structure After Setup
After following the steps, your project should have this structure:

- django_template/
  - package.json
  - tailwind.config.js
  - postcss.config.js
  - static/
    - css/
      - tailwind.css
      - output.css
  - apps/
    - ...
  - config/
    - ...

## Notes
- Tailwind scans files to purge unused CSS—update `content` paths as needed.
- For advanced setups, consider django-tailwind or Vite.

If issues arise, check Node.js version or paths!

## Setting Up Custom Command to Build Tailwind CSS
To streamline the process of building Tailwind CSS, you can create a custom Django management command that runs the Tailwind build process. Here's how to do it:
1. Create a new management command:
   - Inside one of your Django apps (e.g., `core`), create the following directory structure if it doesn't exist:
     ```markdown
     core/
         management/
             commands/
                 __init__.py
                 runserver.py
     ```
2. Implement the command in `runserver.py`:

In [ ]:
# apps/core/management/commands/runserver.py
import subprocess
import sys
from django.core.management.commands.runserver import Command as RunserverCommand

class Command(RunserverCommand):
    def handle(self, *args, **options):
        # Build Tailwind CSS before starting the server
        try:
            print("Building Tailwind CSS...")
            subprocess.run(
                ["bunx","run","build-css-prod"],  # Use prod build for speed; change to build-css for watch
                check=True,
                cwd=self.get_project_root()  # Assumes project root; adjust if needed
            )
            print("Tailwind CSS built successfully.")
        except subprocess.CalledProcessError as e:
            print(f"Error building Tailwind CSS: {e}")
            sys.exit(1)
        
        # Call the original runserver command
        super().handle(*args, **options)
    
    def get_project_root(self):
        # Get the project root (where manage.py is)
        import os
        from pathlib import Path
        return Path(__file__).resolve().parent.parent.parent.parent.parent  # Adjust based on your structure

- If you already don't have a tailwind.css file, create it in static/css/ with the following content:
```css
@tailwind base;
@tailwind components;
@tailwind utilities;
```

### Apps Placement in settings.py for custom command overriding runserver
In your `config/settings.py`, ensure that the app containing your custom command (e.g., `core`) is listed before `django.contrib.staticfiles` in the `INSTALLED_APPS` list. This ensures that your custom command takes precedence over the default `runserver` command.
This is only necessary if you are overriding the `runserver` command specifically.

```python
# Auto Discover apps in the project
discovered_apps = []
APPS_DIR = os.path.join(BASE_DIR, 'apps')
for app_name in os.listdir(APPS_DIR):
    app_path = os.path.join(APPS_DIR, app_name)
    if os.path.isdir(app_path) and os.path.isfile(os.path.join(app_path, '__init__.py')):
        try:
            importlib.import_module(app_name)
            discovered_apps.append(app_name)
        except ImportError as e:
            print(f"Could not import app {app_name}: {e}") 

INSTALLED_APPS = [
    *discovered_apps,
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]
```

## Static Files Configuration
Ensure your `STATIC_URL` is set correctly in `config/settings.py` and tht `STATICFILES_DIRS` includes the static directory.

```python
STATIC_URL = 'static/'

STATICFILES_DIRS = [
    os.path.join(BASE_DIR, 'static'),
]
```
Then to serve static files during development, make sure you have the following in your `config/urls.py` at the bottom of the file:

```python
from django.conf import settings
from django.conf.urls.static import static

if settings.DEBUG:
    urlpatterns += static(settings.STATIC_URL, document_root=settings.STATIC_ROOT)
```
Now you can reference your Tailwind CSS file in your templates using the `{% static %}` template tag:

```html
<link rel="stylesheet" href="{% static 'css/output.css' %}">
```

### Serving Static Files in Production
In a production environment, ensure that your web server (e.g., Nginx, Apache) is configured to serve static files from the directory specified by `STATIC_ROOT`. You may need to run `python manage.py collectstatic` to gather all static files into the `STATIC_ROOT` directory before deploying your application.
- To collect static files for production, run:
```sh
python manage.py collectstatic
```
This will gather all static files into the directory specified by `STATIC_ROOT`, which your web server can then serve.

- Then configure your web server to serve files from the `STATIC_ROOT` directory.
